## 0. Imports

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import LSTM
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dense
import yfinance as yf

In [2]:
#import data
df = pd.read_csv('../../long_short_local/raw_data/cleaned_data.csv')

## 1. Splitting Data in Train and Validation

In [3]:
# R
def split_train_val(df):
    length_data = len(df)    
    split_ratio = 0.7           # %70 train + %30 validation
    length_train = round(length_data * split_ratio)  
    length_validation = length_data - length_train
    #print("Data length :", length_data)
    #print("Train data length:", length_train)
    #print("Validation data lenth:", length_validation)
    
    train_data = df[:length_train].iloc[:,:2] 
    train_data['Date'] = pd.to_datetime(train_data['Date'])  # converting to date time object

    validation_data = df[length_train:].iloc[:,:2]
    validation_data['Date'] = pd.to_datetime(validation_data['Date'])  # converting to date time object
    
    return train_data, validation_data, length_train, length_validation

## 2. Creating Train Dataset from Train split

In [4]:
# R
# create dataset from train split
def train_split(train_data):
    dataset_train = train_data.iloc[:, 1].values
    # Change 1d array to 2d array
    # Changing shape from (1692,) to (1692,1)
    dataset_train = np.reshape(dataset_train, (-1,1))
    #dataset_train.shape
    dataset_train_scaled = dataset_train
    
    return dataset_train_scaled

## 3. Creating X_train and y_train from Train data 

We have train data composed of ratios over days

So, it has 1184 ratios corresponding 1184 days

My aim is to predict the ratio (closing) of the next day.

I can use a time step of 2 days.

I will pick first 2 ratios (day 0 to 2), 1st 2 ratio will be in 
X_train data

Then predict the ratio of 3rd day; and 3rd ratio will be in y_train data

Again, i will pick ratios from 1 to 3, those will be in X_train data

Then predict the next days ratio, 4th ratio will be in y_train data



In [5]:
def create_x_y_train(df, length_train):
    X_train = []
    y_train = []

    time_step = 20 #change that?

    for i in range(time_step, length_train):
        X_train.append(dataset_train_scaled[i-time_step:i,0:1])
        y_train.append(dataset_train_scaled[i,0:1])

    # convert list to array
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    y_train = np.reshape(y_train, (y_train.shape[0],1))
    
    X_train = X_train[:int(X_train.shape[0]*0.95)]
    X_val = X_train[int(X_train.shape[0]*0.95):]
    y_train = y_train[:int(y_train.shape[0]*0.95)]
    y_val = y_train[int(y_train.shape[0]*0.95):]
    
    return X_train, X_val, y_train, y_val, time_step

## 4. Creating Test Dataset from Validation Data

In [6]:
# R
#Converting array and scaling
def create_x_y_test(validation_data, length_validation, time_step):
    dataset_validation = validation_data.iloc[:,1].values  # getting "Ratio" column and converting to array
    dataset_validation = np.reshape(dataset_validation, (-1,1))  # converting 1D to 2D array
    #scaled_dataset_validation =  scaler.fit_transform(dataset_validation)  # scaling  values to between 0 and 1
    scaled_dataset_validation = dataset_validation
    
    X_test = []
    y_test = []

    for i in range(time_step, length_validation):
        X_test.append(scaled_dataset_validation[i-time_step:i,0])
        y_test.append(scaled_dataset_validation[i,0])
        
    # Converting to array
    X_test, y_test = np.array(X_test), np.array(y_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))  # reshape to 3D array
    y_test = np.reshape(y_test, (-1,1))  # reshape to 2D array
    
    return X_test, y_test

## 5. Creating LSTM Model

In [7]:
def lstm_model(X_train, y_train, X_val, y_val, X_test, y_test):
    es = EarlyStopping(patience=20, restore_best_weights=True)
    model_lstm = Sequential()

    model_lstm.add(LSTM(20,return_sequences=False,input_shape = (X_train.shape[1],1))) #64 lstm neuron block
    model_lstm.add(Dense(32))
    model_lstm.add(Dense(1))

    model_lstm.compile(loss = "mape", optimizer = "rmsprop", metrics = ["mae", "mape"])
    history2 = model_lstm.fit(X_train, y_train, epochs = 400, batch_size = 64,validation_data = (X_val, y_val),callbacks=[es])
    mape = model_lstm.evaluate(X_test, y_test)
    
    return mape[2]

## 6. Implemenation refactored

In [66]:
# R
mape_dict = {}

for ratio in df:
   # split into train/test
    if ratio == 'Date':
        continue
    else:
        one_ratio_df = pd.DataFrame(df[['Date', ratio]])
        print_df(one_ratio_df)
        #train_data, validation_data, length_train, length_validation = split_train_val(one_ratio_df)
        # call train_split
        dataset_train_scaled = train_split(train_data)
        # create X_train, y_train
        X_train, X_val, y_train, y_val, time_step = create_x_y_train(dataset_train_scaled, length_train)
        # create X_test, y_test
        X_test, y_test = create_x_y_test(validation_data, length_validation, time_step)
        # run LSTM model
        mape = lstm_model(X_train, y_train, X_val, y_val, X_test, y_test)
        mape_dict[ratio] = round(mape, 3)

           Date  ALB_ZBRA
0    2020-08-31  0.311845
1    2020-09-01  0.332974
2    2020-09-02  0.332368
3    2020-09-03  0.339772
4    2020-09-04  0.356920
..          ...       ...
492  2022-08-15  0.813028
493  2022-08-16  0.814154
494  2022-08-17  0.815450
495  2022-08-18  0.816374
496  2022-08-19  0.822204

[497 rows x 2 columns]
Epoch 1/400
5/5 [==============================] - 1s 80ms/step - loss: 114.5948 - mae: 1.1046 - mape: 114.5948 - val_loss: 71.4919 - val_mae: 0.6832 - val_mape: 71.4919
Epoch 2/400
5/5 [==============================] - 0s 6ms/step - loss: 59.5196 - mae: 0.5724 - mape: 59.5196 - val_loss: 17.0678 - val_mae: 0.1633 - val_mape: 17.0678
Epoch 3/400
5/5 [==============================] - 0s 6ms/step - loss: 10.3096 - mae: 0.1002 - mape: 10.3096 - val_loss: 6.3042 - val_mae: 0.0604 - val_mape: 6.3042
Epoch 4/400
5/5 [==============================] - 0s 7ms/step - loss: 5.7628 - mae: 0.0559 - mape: 5.7628 - val_loss: 3.6407 - val_mae: 0.0346 - val_mape: 3.6407

In [67]:
mape_dict

{'ALB_ZBRA': 2.646,
 'NRG_CPRT': 2.562,
 'DE_BXP': 2.751,
 'NRG_STE': 2.698,
 'NRG_XRAY': 3.351,
 'SO_NLOK': 3.693,
 'PEG_O': 3.438}

In [68]:
print(f'The average MAPE of all {len(mape_dict)} ratios is: {round(sum(mape_dict.values()) / len(mape_dict), 2)}%')

The average MAPE of all 7 ratios is: 3.02%


## 8. Recommend best 5 ratios

In [69]:
# sort from best to worst
bst_list = sorted(mape_dict.items(), key=lambda item: item[1])

In [70]:
best_five = dict(bst_list)
best_five = {k: best_five[k] for k in list(best_five)[:5]}
for k in best_five:
    print(k)

NRG_CPRT
ALB_ZBRA
NRG_STE
DE_BXP
NRG_XRAY


## 9. Compute average true range (ATR)

In [71]:
# once a model is selected based on the lowest mape, we can download open, high, low prices
# convert best 5 to list
best_list = list(best_five.keys())
# get the 5 best ratios from the original dataset
df.set_index('Date', inplace=True)
best_atr_df = df[[c for c in df.columns if c in best_list]]

In [74]:
i = 0
length = best_atr_df.shape[1]
long_list = []
short_list = []

while i < length:
    ratio = best_atr_df.columns.str.split('_').tolist()[i]
    # get the long prices (numerator) for a defined period first
    numerator = ratio[0]
    long = yf.Ticker(numerator).history(period="1mo").reset_index()
    high_low_long = long['High'] - long['Low']
    high_close_long = np.abs(long['High'] - long['Close'].shift())
    low_close_long = np.abs(long['Low'] - long['Close'].shift())
    ranges_long = pd.concat([high_low_long, high_close_long, low_close_long], axis=1)
    true_range_long = np.max(ranges_long, axis=1)
    long[f'{numerator}'] = true_range_long.rolling(14).sum()/14
    long_list.append(long)
    
    # get the short prices (denominator) for a defined period second
    denominator = ratio[1]
    short = yf.Ticker(denominator).history(period="1mo").reset_index()
    high_low_short = short['High'] - short['Low']
    high_close_short = np.abs(short['High'] - short['Close'].shift())
    low_close_short = np.abs(short['Low'] - short['Close'].shift())
    ranges_short = pd.concat([high_low_short, high_close_short, low_close_short], axis=1)
    true_range_short = np.max(ranges_short, axis=1)
    short[f'{denominator}'] = true_range_short.rolling(14).sum()/14
    short_list.append(short)
    
    i += 1

In [75]:
long_df = []
short_df = []

# get list of last columns of long and short ATRs
for long in long_list:
    long_df.append(pd.DataFrame(long[long.columns[-1]]))

for short in short_list:
    short_df.append(pd.DataFrame(short[short.columns[-1]]))  

In [76]:
# generate list of best ratios as tuples
long_short = list(zip(long_df, short_df))

In [77]:
# loop to combine
final_conc_df = pd.DataFrame()
for i in range(0, len(long_short)):
    conc_df = pd.concat([pd.DataFrame(long_short[i][0]), pd.DataFrame(long_short[i][1])], axis=1)
    final_conc_df = pd.concat([final_conc_df, conc_df], axis=1)

In [78]:
# add back 'Date' columns
final_conc_df = pd.concat([final_conc_df, short_list[0]['Date']], axis=1)

In [79]:
# drop NANs
final_conc_df.dropna(how='any', axis=0, inplace=True)
# set index to 'Date' column
final_conc_df.set_index('Date', inplace=True)

In [80]:
# compute ATR for long/short ratios
final_atr_df = pd.DataFrame()
for i in range(0, final_conc_df.shape[1], 2):
    final_atr_df[f'{final_conc_df.columns[i]}_{final_conc_df.columns[i+1]}'] = \
                  final_conc_df.iloc[:, i] / final_conc_df.iloc[:, i+1]

In [81]:
# collect all ATRs in a dict
atr_dict = {}
for col in final_atr_df:
    atr_dict[col] = round(final_atr_df[col].mean(), 3)

In [82]:
atr_dict

{'ALB_ZBRA': 1.008,
 'NRG_CPRT': 0.355,
 'DE_BXP': 5.756,
 'NRG_STE': 0.224,
 'NRG_XRAY': 1.025}